In [2]:
%connect Vantage-LIVE, hidewarnings=true

ERROR: Connection 'Vantage-LIVE' does not exist

In [3]:
%lsconnect

Disconnected: NAME=AWS_APJ, USER=DBC, HOST=APJ-Demo-Perm-NLB4-04313a2144f2d140.elb.ap-southeast-2.amazonaws.com
Disconnected: NAME=AWS_PROD, USER=tb250045, HOST=tdprd2.td.teradata.com, PROPS="logmech=LDAP,tmode=TERA,dbs_port=1025"
Disconnected: NAME=AWS_machine, USER=greig, HOST=13.239.63.137, DATABASE=dbc
Disconnected: NAME=CSE, USER=demo_user, HOST=demo-03p9jzrz3oc8j7qv.env.clearscape.teradata.com
Disconnected: NAME=INDBXSP, USER=tb250045, HOST=tdprd2.td.teradata.com, DATABASE=TRNG_XSP, PROPS="tmode=TERA, logmech=LDAP"
Disconnected: NAME=PROD, USER=tb250045, HOST=tdprd.td.teradata.com, PROPS="logmech=LDAP,tmode=TERA,dbs_port=1025"
Disconnected: NAME=Transcend-Vantage24-Beta, USER=tb250045, HOST=vantage24.td.teradata.com, PROPS="logMech=LDAP,logmech=LDAP"
Disconnected: NAME=Vantage-Live, USER=tb250045, HOST=tdprd3.td.teradata.com, DATABASE=tb250045, PROPS="logmech=LDAP"
Disconnected: NAME=aws_optus, USER=dbc, HOST=141.206.0.153
Disconnected: NAME=aws_optus_mldb, USER=mldb, HOST=141.20

In [6]:
%connect  AWS_PROD

Connected: 'AWS_PROD' connection activated for user 'tb250045'


In [36]:
database ADLDEMOPRD3_tb250045

Success: 1 rows affected

In [9]:
DROP TABLE all_simple_path

Success: 19 rows affected

In [10]:
CREATE TABLE all_simple_path(
    cust_id INTEGER
    ,date_tb DATE FORMAT 'DD/MM/YYYY'
    ,event VARCHAR(10)
)primary index(cust_id)

Success: 0 rows affected

In [11]:
INSERT INTO all_simple_path VALUES(1,'21/06/2022', 'start');
INSERT INTO all_simple_path VALUES(1,'22/06/2022', 'c1');
INSERT INTO all_simple_path VALUES(1,'23/06/2022', 'c3');
INSERT INTO all_simple_path VALUES(1,'24/06/2022', 'conversion');

INSERT INTO all_simple_path VALUES(2,'21/06/2022', 'start');
INSERT INTO all_simple_path VALUES(2,'22/06/2022', 'c1');
INSERT INTO all_simple_path VALUES(2,'23/06/2022', 'c3');
INSERT INTO all_simple_path VALUES(2,'24/06/2022', 'c4');

INSERT INTO all_simple_path VALUES(5,'21/06/2022', 'start');
INSERT INTO all_simple_path VALUES(5,'22/06/2022', 'c1');
INSERT INTO all_simple_path VALUES(5,'24/06/2022', 'c4');
INSERT INTO all_simple_path VALUES(5,'25/06/2022', 'conversion');

INSERT INTO all_simple_path VALUES(3,'21/06/2022', 'start');
INSERT INTO all_simple_path VALUES(3,'22/06/2022', 'c2');
INSERT INTO all_simple_path VALUES(4,'21/06/2022', 'start');
INSERT INTO all_simple_path VALUES(4,'22/06/2022', 'c2');

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

...

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

In [12]:
sel * from all_simple_path order by 1,2

,cust_id,date_tb,event
1,1,2022-06-21,start
2,1,2022-06-22,c1
3,1,2022-06-23,c3
4,1,2022-06-24,conversion
5,2,2022-06-21,start
6,2,2022-06-22,c1
7,2,2022-06-23,c3
8,2,2022-06-24,c4
9,3,2022-06-21,start
10,3,2022-06-22,c2


In [22]:
drop table all_path

Success: 21 rows affected

In [23]:
--Check all the journeys
CREATE table all_path AS(
SELECT * FROM nPath (
ON all_simple_path
  PARTITION BY cust_id
  ORDER BY date_tb
  USING
    Symbols (TRUE AS A)
    Pattern ('A*')
    Mode (NONOVERLAPPING)
    Result (
      FIRST (cust_id OF A) AS cust_id,
      LAST (event OF A) AS last_event,
      ACCUMULATE (event OF A) AS page_path
    )
) AS dt2
) with data primary index(cust_id)


Success: 0 rows affected

In [20]:
SEL * FROM all_path order by cust_id

,cust_id,last_event,page_path
1,1,conversion,"[start, c1, c3, conversion]"
2,2,c4,"[start, c1, c3, c4]"
3,3,c2,"[start, c2]"
4,4,c2,"[start, c2]"
5,5,conversion,"[start, c1, c4, conversion]"


In [ ]:
DROP TABLE pairwise_transistions

In [26]:
--Get the pairwise transistions and mark journeys with and outcome.
CREATE TABLE pairwise_transistions as(
    SELECT dt2.page_path
          ,dt3.last_event
    FROM nPath (
            ON all_simple_path
              PARTITION BY cust_id
              ORDER BY date_tb
              USING
                Symbols (TRUE AS A, TRUE AS B)
                Pattern ('A.B')
                Mode (OVERLAPPING)
                Result (
                  FIRST (cust_id OF A) AS cust_id,
                  ACCUMULATE (event OF ANY(A,B)) AS page_path
                )
            ) AS dt2
    LEFT JOIN 
            all_path as dt3
    ON dt2.cust_id = dt3.cust_id
) WITH DATA PRIMARY INDEX(page_path)

Success: 0 rows affected

In [27]:
--Check pairwise transistions
sel * from pairwise_transistions order by page_path

,page_path,last_event
1,"[c1, c3]",conversion
2,"[c1, c3]",c4
3,"[c1, c4]",conversion
4,"[c3, c4]",c4
5,"[c3, conversion]",conversion
6,"[c4, conversion]",conversion
7,"[start, c1]",conversion
8,"[start, c1]",c4
9,"[start, c2]",c2


In [10]:
--Main program that implements Laplacian smothing and log odds ratios
CREATE TABLE log_odds_example as(
SEL page_path
    ,prob_in_success
    ,prob_in_non_success
    ,ROUND(LOG(prob_in_success/prob_in_non_success),2) as log_odds
FROM(
    SEL COALESCE(a.page_path,b.page_path) as page_path
    ,COALESCE(N_success, 0) as N_success_tb
    ,COALESCE(N_non_success, 0) as N_non_success_tb
    ,c.V
    ,sum(N_success_tb) over(order by a.page_path) as tot_success
    ,sum(N_non_success_tb) over(order by a.page_path) as tot_non_success
    --- Laplacian smothing
    ,(N_success_tb+1.00)/(tot_success+V) as prob_in_success
    ,(N_non_success_tb+1.00)/(tot_non_success+V) as prob_in_non_success
FROM

    (SEL page_path , count(*) as N_success FROM pairwise_transistions where last_event = 'conversion' group by page_path) as a
FULL JOIN 
    (SEL page_path , count(*) as N_non_success FROM pairwise_transistions where last_event <> 'conversion' group by page_path) as b
ON a.page_path = b.page_path
LEFT OUTER JOIN
    (SEL count(distinct page_path)as V  FROM pairwise_transistions ) as c
ON 1=1
) as d
) with data primary index(page_path);

Success: 0 rows affected

In [11]:
SEL * FROM log_odds_example order by page_path

,page_path,prob_in_success,prob_in_non_success,log_odds
1,"[c1, c3]",.17,.18,-0.03
2,"[c1, c4]",.17,.09,0.28
3,"[c3, c4]",.08,.18,-0.36
4,"[c3, conversion]",.17,.09,0.28
5,"[c4, conversion]",.17,.09,0.28
6,"[start, c1]",.17,.18,-0.03
7,"[start, c2]",.08,.18,-0.36


In [22]:
DROP TABLE new_journey;

Success: 16 rows affected

In [23]:
-- Scoring new journey. 
-- Note I will skip N_path to get pair wise transistions for a new journey. 
-- The code is the same as for creating a model
CREATE TABLE new_journey(
    page_path VARCHAR(20)
    ,cust_id INTEGER
)primary index(cust_id)

Success: 0 rows affected

In [24]:
INSERT INTO new_journey VALUES('[start, c1]',6);
INSERT INTO new_journey VALUES('[c1, c3]',6);
INSERT INTO new_journey VALUES('[c3, c4]',6);

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

In [25]:
SEL * FROM new_journey

,page_path,cust_id
1,"[start, c1]",6
2,"[c1, c3]",6
3,"[c3, c4]",6


In [37]:
--Score a journey 
SEL a.*,b.*
   ,ROUND(sum(b.log_odds) over(order by a.page_path),2) AS log_likelyhood_to_convert
FROM 
    new_journey as a
left join
    log_odds_example as b
ON a.page_path = b.page_path

,page_path,cust_id,page_path-1,prob_in_success,prob_in_non_success,log_odds,log_likelyhood
1,"[c1, c3]",6,"[c1, c3]",.17,.18,-0.03,-0.42
2,"[c3, c4]",6,"[c3, c4]",.08,.18,-0.36,-0.42
3,"[start, c1]",6,"[start, c1]",.17,.18,-0.03,-0.42
